Here, geolocalization is added to the neighborhoods. First, let's repeat all the steps from part 1 of the task to reproduce the dataframe without geodata.

In [1]:
from bs4 import BeautifulSoup #for page scraping
import pandas as pd #for dataframe functionality
import requests #for getting the page
import re #for regular expressions
import numpy as np #for data manipulation functionality
import geocoder #for geocoding

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_response = requests.get(link, timeout=10)
content = BeautifulSoup(page_response.content, "html.parser")

table = content.find('table')
rows = table.find_all('tr')
temp = []
for tr in rows:
    td = tr.find_all('td')
    row = [tr.get_text(strip=True) for tr in td]
    temp.append(row)
    
df=pd.DataFrame(temp, columns=["PostalCode", "Borough","Neighborhood"])
df = df.drop([0],axis=0)
df = df[df.Borough != 'Not assigned']
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df=df.groupby(['PostalCode','Borough'],as_index=False).agg(', '.join)

Due to issues with Geocoder and different providers of geodata that I tried, I use the file with the coordinates instead.

In [3]:
geodata_url="http://cocl.us/Geospatial_data"
df_geo=pd.read_csv(geodata_url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, the dataframe with geodata is joined with the original.

In [4]:
df = df.join(df_geo.set_index('Postal Code'), on='PostalCode')
df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
